In [1]:
###### import
import numpy as np
import random
import matplotlib.pyplot as plt
import matplotlib.cm as cm 
import seaborn as sns
from scipy import signal
from importlib import reload

import winsound as sd
import pickle
import sys
import os

from Tools.tools import *
from Plasticity.plasticity import *
from Neuron.neuron import *
from Model.model import *
from Analysis.analysis import *
import Operate_model_v4_fig

In [2]:
# Load
cd = os.getcwd()
path = os.path.dirname(cd)

with open(path+'//Data_final16_v2.pkl', 'rb') as file:
    # Patterns in input layer
    Data = pickle.load(file)   
    # The number of data
    Datalen = pickle.load(file)    
    # Patterns of granule cells
    Data_DG = pickle.load(file)
    
    # Lengths of each patterns of granule cells
    Data_DG_len = pickle.load(file)
    Data_idx = pickle.load(file) # indices of patterns having proper lengths.
    DataX_idx = pickle.load(file) # indices of patterns having proper lengths.
    
    # 각 pair에 대한 input/output overlap값과 index
    OverI = pickle.load(file)
    OverO = pickle.load(file)
    Pair_idx = pickle.load(file)
    
    # 길이로 분류한 pair에 대한 input/output overlap값과 index
    OverI_v2 = pickle.load(file)
    OverO_v2 = pickle.load(file)
    Pair_idx_v2 = pickle.load(file)
    
with open(path+'//Data_final16_v2-CA3.pkl', 'rb') as file:
    # Filtered by proper length
    Filt_idata = pickle.load(file) # patterns in input layer
    Filt_gdata = pickle.load(file) # patterns of granule cells
    Filt_glen = pickle.load(file) # lengths of each patterns of granule cells
    F_DataLen = pickle.load(file) # the number of data
    
    # Index of CA3 patterns
    Af_py = pickle.load(file) # Pyramidal neurons
    Af_in = pickle.load(file) # Inhibitory neurons
    
    # 뭐였더라
    In_py_list = pickle.load(file)
    In_in_list = pickle.load(file)

In [3]:
# Classify CA3 patterns by length 9-14
CA3_size = []
for n in range(F_DataLen):
    CA3_size.append(len(Af_py[n]))

# Classified by size
CA3_SC = [[],[],[],[],[],[]]
size_min = 9
size_max = size_min+len(CA3_SC)
for i, comp in enumerate(CA3_size):
    if size_min <= comp < size_max:
        idx = comp-size_min
        CA3_SC[idx].append(i)

Number = 0
for n in range(len(CA3_SC)):
    Number += len(CA3_SC[n])
print(Number)

Af_py_final_idx = set()
for n in range(len(CA3_SC)):
    Af_py_final_idx |= set(CA3_SC[n])

332


In [5]:
# Classify CA3 patterns by overlap degree
CA3_OC = []
Classfy_num = 20
for n in range(Classfy_num+1):
    CA3_OC.append([])

input_total = set(range(2400))
for i, comp in enumerate(Af_py):
    for j, comp2 in enumerate(Af_py):
        if size_min <= len(comp) < size_max:
            if size_min <= len(comp2) < size_max:
                if i < j:
                    temp = cal_overlap(input_total, set(comp), set(comp2))
                    if temp == 0:
                        CA3_OC[0].append([i,j])
                    for n in range(9):
                        if 0.05*n <= temp < 0.05*n+0.05: 
                            CA3_OC[n+1].append([i,j])

In [6]:
# Classify CA3 patterns by both
CA3_SOC = []
for i, comp in enumerate(CA3_OC[0]):
    for j, comp2 in enumerate(CA3_SC):
        if comp[0] in comp2:
            temp0 = j
        if comp[1] in comp2:
            temp1 = j
    # 같은 size 정보면
    if temp0 == temp1:
        CA3_SOC.append(comp)